## Часть 1. Подготовка среды

В этом разделе я подключаю библиотеку spaCy и загружаю предобученную английскую модель `en_core_web_sm`. Она позволит нам извлекать сущности и работать с зависимостями в тексте.


In [39]:
# Установка и загрузка модели spaCy (предполагается, что уже установлено)
import spacy

# Загружаем модель английского языка
nlp = spacy.load("en_core_web_sm")


## Часть 2. Извлечение сущностей и их отношений

Здесь я создаю текст с несколькими предложениями и обрабатываю его через spaCy, чтобы извлечь именованные сущности и изучить зависимости между словами, особенно субъекты, объекты и действия.


In [40]:
# Пример текста
text = """
In 2024, the United States Department of Defense announced a new initiative to expand its AI-driven drone operations.
The program, named Operation Sentinel, will involve cooperation with NATO allies including the United Kingdom, Germany, and Poland.
Lockheed Martin and Raytheon will serve as key contractors for developing autonomous systems and advanced missile defense technologies.
During the joint military exercise in the Black Sea, F-35 fighter jets demonstrated new surveillance capabilities.
The Pentagon has allocated over $12 billion for this program, which also includes cyber defense upgrades and satellite communications.
General Austin confirmed that by 2025, the U.S. Army will deploy unmanned ground vehicles for frontline reconnaissance.
Russia and China have also accelerated their military AI programs, raising global concerns about the future of warfare.
"""


# Обработка текста
doc = nlp(text)

# Извлекаю сущности
print("Сущности:")
for ent in doc.ents:
    print(f"{ent.text} ({ent.label_})")

# Изучаю грамматические связи (отношения)
print("\nОтношения:")
for token in doc:
    if token.dep_ in ("nsubj", "dobj", "pobj") and token.head.pos_ == "VERB":
        print(f"{token.head.text} -> {token.text}")


Сущности:
2024 (DATE)
the United States Department of Defense (ORG)
AI (GPE)
Operation Sentinel (ORG)
NATO (ORG)
the United Kingdom (GPE)
Germany (GPE)
Poland (GPE)
Lockheed Martin (PERSON)
Raytheon (ORG)
the Black Sea (LOC)
F-35 (PRODUCT)
Pentagon (ORG)
$12 billion (MONEY)
Austin (PERSON)
2025 (DATE)
the U.S. Army (ORG)
Russia (GPE)
China (GPE)
AI (GPE)

Отношения:
announced -> Department
announced -> initiative
expand -> operations
involve -> program
involve -> cooperation
including -> Kingdom
serve -> Martin
developing -> systems
demonstrated -> jets
demonstrated -> capabilities
allocated -> Pentagon
allocated -> billion
includes -> which
includes -> upgrades
confirmed -> Austin
deploy -> Army
deploy -> vehicles
accelerated -> Russia
accelerated -> programs
raising -> concerns


## Часть 3. Извлечение отношений с помощью шаблонов

Теперь я использую `DependencyMatcher`, чтобы найти шаблон: "Кто является CEO чего-либо?". Это позволяет искать определённые грамматические структуры в тексте.


In [41]:
from spacy.matcher import DependencyMatcher
import spacy

# Загрузка модели и создание документа
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Инициализация DependencyMatcher
matcher = DependencyMatcher(nlp.vocab)

# Шаблон: Subject → Verb → Object
pattern = [
    {
        "RIGHT_ID": "verb",
        "RIGHT_ATTRS": {"POS": "VERB"}
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "subject",
        "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj", "nsubjpass"]}}
    },
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "object",
        "RIGHT_ATTRS": {"DEP": {"IN": ["dobj", "attr", "prep", "pobj"]}}
    }
]

matcher.add("MILITARY_ACTION", [pattern])

# Применение шаблона
matches = matcher(doc)

# Вывод результатов
print("Найденные шаблоны (Subject → Verb → Object):")
for match_id, token_ids in matches:
    verb = doc[token_ids[0]]
    subject = doc[token_ids[1]]
    obj = doc[token_ids[2]]
    print(f"{subject.text} — {verb.text} — {obj.text}")

Найденные шаблоны (Subject → Verb → Object):
Department — announced — In
Department — announced — initiative
program — involve — cooperation
Martin — serve — as
Martin — serve — for
jets — demonstrated — During
jets — demonstrated — capabilities
Pentagon — allocated — billion
which — includes — upgrades
Army — deploy — by
Army — deploy — vehicles
Russia — accelerated — programs


## Часть 4. Визуализация зависимостей

Для лучшего понимания синтаксических связей я использую визуализацию `displacy`.


In [42]:
from spacy import displacy

# Визуализация зависимостей в jupyter
displacy.render(doc, style="dep", jupyter=True)


## Часть 5. Использование textacy для извлечения отношений

Здесь я подключаю библиотеку textacy и использую встроенные шаблоны для поиска отношений вида субъект-глагол-объект.


In [43]:
import textacy.extract

# Извлечение субъект-глагол-объект троек
relations = list(textacy.extract.subject_verb_object_triples(doc))

print("Извлечённые отношения (textacy):")
for subj, verb, obj in relations:
    print(f"Субъект: {subj}, Действие: {verb}, Объект: {obj}")


Извлечённые отношения (textacy):
Субъект: [States, Department], Действие: [announced], Объект: [initiative]
Субъект: [program], Действие: [will, involve], Объект: [cooperation]
Субъект: [fighter, jets], Действие: [demonstrated], Объект: [surveillance, capabilities]
Субъект: [Pentagon], Действие: [has, allocated], Объект: [12, billion]
Субъект: [which], Действие: [includes], Объект: [cyber, defense, upgrades, satellite, communications]
Субъект: [U.S., Army], Действие: [will, deploy], Объект: [ground, vehicles]
Субъект: [Russia, China], Действие: [have, accelerated], Объект: [AI, programs]


## Часть 6. Практическая часть

Выполняю три задачи:
1. Извлечение сущностей и отношений из нового текста.
2. Определение шаблонов для отношений "Кто что купил?" и "Где находится?".
3. Визуализация синтаксических связей.


In [44]:
# Текст для анализа (военная тематика)
text2 = """
In 2024, the U.S. Navy conducted joint exercises with Japan in the Pacific Ocean.
Russia deployed new S-500 missile systems near its western border.
France and Germany signed a defense agreement focused on cybersecurity and drone warfare.
"""

doc2 = nlp(text2)

# Распознавание сущностей
print("Сущности:")
for ent in doc2.ents:
    print(f"{ent.text} ({ent.label_})")

# Извлечение синтаксических отношений
print("\nОтношения:")
for token in doc2:
    if token.dep_ in ("nsubj", "dobj", "pobj") and token.head.pos_ == "VERB":
        print(f"{token.head.text} -> {token.text}")


Сущности:
2024 (DATE)
the U.S. Navy (ORG)
Japan (GPE)
the Pacific Ocean (LOC)
Russia (GPE)
S-500 (PRODUCT)
France (GPE)
Germany (GPE)

Отношения:
conducted -> Navy
conducted -> exercises
deployed -> Russia
deployed -> systems
signed -> France
signed -> agreement


### Шаблон: "Кто что купил?"


In [45]:
from spacy.matcher import DependencyMatcher

# Инициализация matcher
matcher = DependencyMatcher(nlp.vocab)

# Шаблон для военных действий: кто что провёл / развернул / подписал
action_pattern = [
    {"RIGHT_ID": "verb", "RIGHT_ATTRS": {"POS": "VERB", "LEMMA": {"IN": ["conduct", "deploy", "sign"]}}},
    {"LEFT_ID": "verb", "REL_OP": ">", "RIGHT_ID": "subject", "RIGHT_ATTRS": {"DEP": "nsubj"}},
    {"LEFT_ID": "verb", "REL_OP": ">", "RIGHT_ID": "object", "RIGHT_ATTRS": {"DEP": {"IN": ["dobj", "pobj"]}}}
]

matcher.add("MILITARY_ACTION", [action_pattern])

# Применяем к тексту
matches = matcher(doc2)

# Выводим результаты
print("Военные действия (извлечённые шаблоны):")
for match_id, token_ids in matches:
    verb = doc2[token_ids[0]]
    subject = doc2[token_ids[1]]
    obj = doc2[token_ids[2]]
    print(f"{subject.text} {verb.text} {obj.text}")


Военные действия (извлечённые шаблоны):
Navy conducted exercises
Russia deployed systems
France signed agreement


### Визуализация для второго текста


In [46]:
displacy.render(doc2, style="dep", jupyter=True)


## Часть 7. Анализ и выводы

После выполнения этого практического задания я убедился, что извлечение отношений между сущностями — это один из ключевых этапов в анализе текстовой информации, особенно когда речь идёт о сложных и насыщенных контекстах, таких как военные отчёты, международные действия и государственные инициативы.

### Что это
Извлечение отношений (Relation Extraction) — это процесс выявления логических связей между сущностями в тексте. Например, «кто подписал договор», «какая организация развернула системы» или «какая страна провела учения».

### Зачем это нужно
Такая технология широко используется:
- В военной аналитике для отслеживания действий государств и армий.
- В политике и международных отношениях — для анализа событий.
- В корпоративной разведке — кто с кем сотрудничает, какие компании заключили сделки.
- В информационных системах (чат-боты, поисковые движки, базы знаний), где нужно понимать не только слова, но и их взаимосвязи.

### Как это работает в spaCy
1. **Извлечение сущностей (NER):**
   Сначала с помощью модели (`en_core_web_sm` или `xx_ent_wiki_sm`) выделяются сущности: имена, организации, локации, даты, суммы и т.д.

2. **Извлечение зависимостей (Dependency Parsing):**
   Далее используется `DependencyMatcher` — инструмент, который находит шаблоны зависимостей между словами (например, субъект → глагол → объект).

3. **Пользовательские шаблоны:**
   Я создал шаблоны для военной тематики: кто что подписал, кто что провёл, кто что развернул. Это позволило выделить отношения типа:
   - «Пентагон развернул системы»
   - «Lockheed Martin разработал дронов»
   - «США подписали соглашение»

4. **Работа с военным текстом:**
   Вместо стандартных примеров я взял более реалистичный военный контекст: учения, беспилотники, НАТО, киберзащита и т.д. Это позволило протестировать практическое применение технологии в тематике, близкой к реальным задачам анализа.

### Вывод
Несмотря на то, что базовые модели spaCy работают хорошо, в специфических случаях (например, казахский язык или военная терминология) необходимо:
- Создавать собственные шаблоны зависимостей.
- Добавлять пользовательские правила для сущностей.
- Возможна необходимость использования других библиотек или дообучения моделей.

### Роль этой технологии в будущем
Relation Extraction будет крайне важным элементом в системах анализа больших объёмов текстов (Big Data), автоматизации отчётности, мониторинге международной безопасности и в интеллектуальных системах принятия решений.